![alt text](./Cerny_logo_1.jpg)

# Analysis of Cerny ventilation recordings

The data processed and analysed in this Notebook were collected by the **Neonatal Emergency and Transport Service of the Peter Cerny Foundation**, Budapest, Hungary

**Author: Dr Gusztav Belteki**


## Processing of clinical data for recordings `AL000001 - AL000300`

This notebook processes the **clinical data** for ventilation recordings `AL000001 - AL000300`

- Total: **300 cases**
- Clinical data available in **259 cases**
- Only keep clinical data for cases where ventilation recordings (>15 minutes) are also available: **242 cases**

A DataFrame containing the processed clinical data is exported as **clin_df_1_300.pickle**

### Importing the necessary libraries and setting options

In [ ]:
import IPython
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk

import os
import sys
import re
import pickle

from scipy import stats
from pandas import Series, DataFrame
from datetime import datetime, timedelta

%matplotlib inline
matplotlib.style.use('classic')
matplotlib.rcParams['figure.facecolor'] = 'w'

pd.set_option('display.max_rows', 250)
pd.set_option('display.max_columns', 250)
# pd.set_option('mode.chained_assignment', None) 

In [ ]:
print("Python version: {}".format(sys.version))
print("pandas version: {}".format(pd.__version__))
print("matplotlib version: {}".format(matplotlib.__version__))
print("NumPy version: {}".format(np.__version__))
print("SciPy version: {}".format(sp.__version__))
print("IPython version: {}".format(IPython.__version__))
print("scikit-learn version: {}".format(sk.__version__))

### List and set the working directory and the directory to write out data

In [ ]:
# Topic of the Notebook which will also be the name of the subfolder containing results
TOPIC = 'fabian'

# Name of the external hard drive
DRIVE = 'GUSZTI'

# Directory containing clinical and blood gas data
CWD = '/Users/guszti/ventilation_fabian'

# Directory on external drive to read the ventilation data from
DIR_READ = '/Volumes/%s/Fabian/fabian_patient_data' % DRIVE

DIR_WRITE = '%s/%s' % (CWD, 'Analyses')

# Images and raw data will be written on an external hard drive
if not os.path.isdir('/Volumes/%s/data_dump/%s' % (DRIVE, TOPIC)):
    os.makedirs('/Volumes/%s/data_dump/%s' % (DRIVE, TOPIC))
DATA_DUMP = '/Volumes/%s/data_dump/%s' % (DRIVE, TOPIC)

In [ ]:
os.chdir(CWD)
os.getcwd()

In [ ]:
DIR_READ

In [ ]:
DIR_WRITE

In [ ]:
DATA_DUMP

### Import ventilation data

This is needed to know the beginning and the end of the recordings

In [ ]:
with open('%s/%s.pickle' % (DATA_DUMP, 'data_pars_1_150'), 'rb') as handle:
    data_pars_1_150 = pickle.load(handle)

with open('%s/%s.pickle' % (DATA_DUMP, 'data_pars_151_300'), 'rb') as handle:
    data_pars_151_300 = pickle.load(handle)
    
data_pars = {**data_pars_1_150, **data_pars_151_300}

In [ ]:
len(data_pars)

### Import clinical data

In [ ]:
# import text files in a dictionary
clin_dict = {}
for fname in os.listdir(DIR_READ):
    if not fname.startswith('.'): # disregard hidden files
        fhandle = open(os.path.join('%s' % DIR_READ, fname), 'r', encoding = 'cp1252')
        clin_dict[fname[:-4]] = fhandle.read() # use the filenames without the .txt extension as keys
        fhandle.close()

In [ ]:
# split the clinical data into a list
for key in sorted(clin_dict.keys()):
    clin_dict[key] = clin_dict[key].split('\n')[:-1]

In [ ]:
# Create an inner dictionary for the different clinical data
for key, value in sorted(clin_dict.items()):
    temp_dict = {}
    for item in value:
        td_key, *td_value = item.split(':')
        td_key = td_key.strip()
        temp_dict[td_key] = ''.join(td_value)[1:]
    clin_dict[key] = temp_dict

In [ ]:
# Create a DataFrame from the dictionary of dictionaries
clin_df = DataFrame(clin_dict).T
clin_df.index.name = 'Recording_ID'
clin_df.sort_index(inplace = True)

In [ ]:
len(clin_df)

In [ ]:
clin_df;

### Limit it to the first 300 cases

In [ ]:
clin_df = clin_df.iloc[:-107]

In [ ]:
clin_df.tail()

### Drop cases which have no clinical data

In [ ]:
clin_df.dropna(axis = 0, how = 'all', inplace = True)

In [ ]:
len(clin_df)

### Drop cases for which there is no ventilation data

Ventilation recordings may have been excluded because they were two short (<15 mintes total) or aberrant

In [ ]:
combined = sorted(set(list(clin_df.index)) & set(data_pars.keys()))

In [ ]:
clin_df = clin_df.loc[combined]
len(clin_df)

### Clean up clinical dataframe

In [ ]:
clin_df.head()

In [ ]:
# Change order of columns and create English names

clin_df = clin_df[['Esetlap id', 'Date of Birth', 'Gestation Age', 'Birth Weight', 
                   'Actual Weight', 'Pathology', 'Start', 'End']]
clin_df.columns = ['Case ID', 'Date of Birth', 'Gestational Age',
                   'Birth Weight', 'Actual Weight', 'Pathology', 'Start', 'End']

In [ ]:
clin_df['Gestational Age'] = clin_df['Gestational Age'].map(lambda x: int(x[:2]))
clin_df['Birth Weight'] = clin_df['Birth Weight'].map(lambda x: int(x[:-6]))
clin_df['Actual Weight'] = clin_df['Actual Weight'].str.strip(' grams')

In [ ]:
actual_weight = []
for i in range(len(clin_df)):
    if clin_df.iloc[i]['Actual Weight'] == '':
        actual_weight.append(clin_df.iloc[i]['Birth Weight'])
    else:
        actual_weight.append(int(clin_df.iloc[i]['Actual Weight']))

clin_df['Weight'] = actual_weight

#### Start and end from ventilation data

In [ ]:
starts = {}; ends = {}
for rec in sorted(clin_df.index):
    try:
        starts[rec] = data_pars[rec].index[0]
    except KeyError:
        continue
        
    try:
        ends[rec] = data_pars[rec].index[-1]
    except KeyError:
        continue
        
start_end = DataFrame([starts, ends]).T
start_end.columns = ['Recording start', 'Recording end']

In [ ]:
clin_df = pd.concat([clin_df, start_end], axis = 1, join = 'outer')

In [ ]:
clin_df['Date of Birth'] = clin_df['Date of Birth'].map(lambda x: pd.to_datetime(x))
clin_df['Pathology'] = clin_df['Pathology'].map(lambda x: x.split(';')[:-1])

In [ ]:
clin_df['Duration'] = clin_df['Recording end'] - clin_df['Recording start']

In [ ]:
clin_df['Postnatal Age'] = clin_df['Recording end'] - clin_df['Date of Birth']

In [ ]:
clin_df.info()

In [ ]:
clin_df['Gestational Age'] = pd.to_timedelta((clin_df['Gestational Age']), unit='W', 
                                                           box=True, errors='raise')

In [ ]:
clin_df['Corrected gestational Age'] = pd.to_timedelta((clin_df['Gestational Age']), unit='D', 
                          box=True, errors='raise') + clin_df['Postnatal Age']

In [ ]:
clin_df['Gestational Age (weeks)'] = \
    clin_df['Gestational Age'].apply(lambda x: x.total_seconds() / (60 * 60 * 24 * 7))

clin_df['Corrected gestational Age (weeks)'] = \
    clin_df['Corrected gestational Age'].apply(lambda x: round(x.total_seconds() / (60 * 60 * 24 * 7), 1))

In [ ]:
clin_df.sort_index(axis = 1).head()

### EDA on clinical details

In [ ]:
clin_df.describe()

#### For some recordings the age at the time of transfer is "negative"  - these need to be corrected

In [ ]:
clin_df[clin_df['Postnatal Age'] < pd.to_timedelta(0)]

#### For some recordings the duration of the recording is "negative"  - these need to be corrected

In [ ]:
clin_df[clin_df['Duration'] < pd.to_timedelta(0)]

#### Babies was at less than 23 weeks gestation

In [ ]:
clin_df[clin_df['Gestational Age (weeks)'] < 23]

#### Babies born with less than 500 g birth weight

In [ ]:
clin_df[clin_df['Birth Weight'] < 500]

In [ ]:
len(clin_df[clin_df['Birth Weight'] < 500])

#### Babies transferred with the postnatal age of > 46 weeks we need to discuss whether to include them in the data analysis

In [ ]:
a = clin_df[clin_df['Corrected gestational Age (weeks)'] > 46]
a.sort_values('Corrected gestational Age (weeks)')

In [ ]:
len(clin_df[clin_df['Corrected gestational Age (weeks)'] > 46])

### Identify ICD codes

In [ ]:
icd = re.compile(r'(\S\d+\.\d+)')

In [ ]:
def icd_finder(lst):
    icd_list = []
    for item in lst:
        if icd.findall(item):
            icd_list.append(icd.findall(item)[0])
    
    return icd_list

In [ ]:
def icd_cleanup(lst):
    icd_list = []
    for item in lst:
        new_item = item[0 : item.index('.')] + item[item.index('.') + 1 : ]
        icd_list.append(new_item)
    return icd_list

In [ ]:
clin_df['ICD'] = clin_df['Pathology'].apply(icd_finder)

In [ ]:
clin_df['ICD'] = clin_df['ICD'].apply(icd_cleanup)

In [ ]:
clin_df.head()

### Identify all icd codes

In [ ]:
icd_all = []

for _, item in clin_df.iterrows():
    icd_all.extend(item['ICD'])
    
icd_all = sorted(set(icd_all))

In [ ]:
# All ICD codes from AL000001 - AL000457
print(icd_all)

In [ ]:
len(icd_all)

In [ ]:
icd_all_frme = DataFrame(icd_all)
icd_all_frme.columns = ['code']
icd_all_frme.sort_values(by = 'code', inplace = True)

In [ ]:
icd_all_frme.head()

In [ ]:
len(icd_all_frme)

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'icd_codes.xlsx'))
icd_all_frme.to_excel(writer, 'icd_codes')
writer.save()

### Create Pathology column with English names

Import the file that has been manually curate from the exported `icd_codes.xlsx` files to contain all relevant diagnosis

In [ ]:
icd_codes = pd.read_excel('/Users/guszti/ventilation_fabian/icd_codes_curated.xlsx', 
                          usecols = [0,1], index_col = 0)

In [ ]:
icd_codes.head()

In [ ]:
icd_dictionary = dict(zip(icd_codes.index, icd_codes['name']))

In [ ]:
def icd_replace(lst):
    icd_list = []
    for item in lst:
        new_item = icd_dictionary[item]
        icd_list.append(new_item)
    return icd_list

In [ ]:
clin_df['Pathology_English'] = clin_df['ICD'].apply(icd_replace)

### Final cleanup of the DataFrame

In [ ]:
clin_df.columns

In [ ]:
column_list = ['Case ID', 'Date of Birth', 'Gestational Age (weeks)', 'Birth Weight',
              'Postnatal Age', 'Corrected gestational Age (weeks)',  'Weight',
              'ICD', 'Pathology_English', 'Recording start', 'Recording end', 'Duration',]
                    
clin_df = clin_df[column_list]

### Statistics on clinical data

In [ ]:
clinical_stats = round(clin_df.describe(percentiles = [0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]), 1)
clinical_stats

In [ ]:
clin_df.head()

### Export clinical information as an Excel sheet

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'clinical_data_all_1_300.xlsx'))
clin_df.to_excel(writer, 'clin_df')
writer.save()

### Export statistics on clinical data

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'clinical_stats_1_300.xlsx'))
clinical_stats.to_excel(writer, 'stats')
writer.save()

### Export processed data as pickle files

In [ ]:
with open('%s/%s.pickle' % (DATA_DUMP, 'clin_df_1_300'), 'wb') as handle:
    pickle.dump(clin_df, handle, protocol=pickle.HIGHEST_PROTOCOL)